In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import requests
from os import path as osp
import json
import regex as re  # required since regular re does not support character classes \p
import glob
from tqdm import tqdm


torch.backends.cudnn.benchmark = True
device = "cuda:0" if torch.cuda.is_available() else "cpu"

from data import *
from network import *
from train import *

%load_ext autoreload
%autoreload 1
%aimport data,network,train

In [ ]:
enc = get_encoder()
idx_list = enc.encode("Yo what up, that's so call! Indubitably, albeit that's incomprehensively not watto strengthening my resolve?")
print(idx_list)
print(enc.decode(idx_list))
print(enc.cache)

In [ ]:
#
eng_files = glob.glob("1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/*")
np.random.shuffle(eng_files)
enc.encode_file_list("1-billion-word-language-modeling-benchmark-r13output/encoded.bin", eng_files)

In [ ]:
train_dataset = TextDataset(lines)
train_loader = DataLoader(train_dataset, batch_size=16,
                          sampler=torch.utils.data.RandomSampler(train_dataset, replacement=True),
                          collate_fn=train_dataset.collate_fn,
                          pin_memory=True,
                          num_workers=7)
print(len(train_loader))
#test_net = Transformer(train_dataset.vocab_size, 2, 120, 5, 512).to(device)


In [ ]:
lens = []
for t in tqdm(train_loader):
    lens.append(t[0].shape[1])

In [ ]:
bin_edges = "auto" #list(range(5,50,5)) + [100, 200, 300, 5000]
print(bin_edges)
#np.histogram(lens, bins=bin_edges)
plt.hist(lens, bins=bin_edges)
plt.gca().set_yscale("log")
plt.gca().set_xscale("log")